In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"


import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [2]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Input, Flatten,\
Conv2DTranspose, BatchNormalization, LeakyReLU, Reshape, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import cifar10
import tensorflow.keras.backend as K
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner
import time
# import plotly
# import plotly.express as px

In [3]:
# input image dimensions
img_rows, img_cols = 32, 32
img_chn = 1

# Load CIFAR-10 dataset-
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
if img_chn == 1:
    x_train = x_train.mean(axis=3)                                                                                    
    x_test = x_test.mean(axis=3)           
    
input_shape = (img_rows, img_cols, img_chn)

# Convert datasets to floating point types-
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize the training and testing datasets-
if img_chn == 1:
    x_train = x_train.reshape((x_train.shape[0], \
                                         img_rows, img_rows, img_chn)) / 255.
    x_test = x_test.reshape((x_test.shape[0], \
                                      img_rows, img_rows, img_chn)) / 255.
else:
    x_train = x_train/255.
    x_test = x_test/255.


print("\nDimensions of training and testing sets are:")
print(f"X_train.shape: {x_train.shape} & X_test.shape: {x_test.shape}")

load_outlier_detector = True

latent_dim = 2
epochs = 20


Dimensions of training and testing sets are:
X_train.shape: (50000, 32, 32, 1) & X_test.shape: (10000, 32, 32, 1)


In [4]:
def batch_creation(X_data, batch_size):
    end = X_data.shape[0]-1
    start = 0
    batch_split_X = list()
    while start < end:
        img_slice = np.array(X_data[start:start+batch_size])
        batch_split_X.append(img_slice)
        start = start + batch_size
    return np.array(batch_split_X, dtype=object)
    
train_dataset = batch_creation(x_train, 32)
test_dataset = batch_creation(x_test, 32)
train_dataset.shape

(1563,)

In [5]:
def plot_latent_images(model, n, epoch, im_size=32, save=True, first_epoch=False, f_ep_count=0):
    
    # Create image matrix 
    image_width = im_size*n
    image_height = image_width
    image = np.zeros((image_height, image_width, img_chn))

    # Create list of values which are evenly spaced wrt probability mass

    norm = tfp.distributions.Normal(0, 1)
    grid_x = norm.quantile(np.linspace(0.05, 0.95, n))
    grid_y = norm.quantile(np.linspace(0.05, 0.95, n))
    
    # For each point on the grid in the latent space, decode and

    # copy the image into the image array
    for i, yi in enumerate(grid_x):
        for j, xi in enumerate(grid_y):
            z = np.array([[xi, yi]])
            x_decoded = model.sample(z)
            digit = tf.reshape(x_decoded[0], (im_size, im_size, img_chn))
            image[i * im_size: (i + 1) * im_size,
                  j * im_size: (j + 1) * im_size] = digit.numpy()
    

    # Plot the image array
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap='Greys_r')
    plt.axis('Off')


#     # Potentially save, with different formatting if within first epoch
#     if save and first_epoch:
#         plt.savefig('tf_grid_at_epoch_{:04d}.{:04d}.png'.format(epoch, f_ep_count))
#     elif save:
#         plt.savefig('tf_grid_at_epoch_{:04d}.png'.format(epoch))
#     plt.show()

In [6]:

def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(
        -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
        axis=raxis)

def compute_loss(model, x):
    mean, logvar, z = model.encode(x)
    print(f'In compute loss')
    x_logit = model.decode(z)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
#     print(f'Ending compute loss, {-tf.reduce_mean(logpx_z + logpz - logqz_x)}')
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

In [7]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
latent_dim = 2
# input image dimensions
img_rows, img_cols = 32, 32
img_chn = 1

In [8]:
# m.summary()

In [9]:
class Encoder(tf.keras.Model):
    def __init__(self,hp):
        super(Encoder, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),\
        #activation function
        activation='relu',input_shape=(32,32,1))
        self.conv2 = tf.keras.layers.Conv2D(64, 3, strides=2, padding='same', activation=tf.nn.relu)
        self.conv3 = tf.keras.layers.Conv2D(128, 3, strides=2, padding='same', activation=tf.nn.relu)
        self.flat1 = tf.keras.layers.Flatten()
        self.dens1 = tf.keras.layers.Dense(2)
        self.dens2 = tf.keras.layers.Dense(2)
        self.sampl = Sampling()
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flat1(x)
        mu = self.dens1(x)
        va = self.dens2(x)
        z = self.sampl([mu,va])
        
        return mu, va, z
    
    def model(self):
        x = Input(shape=(32,32,1))
        return Model(inputs=[x], outputs=self.call(x))
    
class Decoder(tf.keras.Model):
    def __init__(self, ):
        super(Decoder, self).__init__()
        self.dens3 = tf.keras.layers.Dense(units=4*4*128, activation=tf.nn.relu, input_shape=(2,))
        self.resh1 = tf.keras.layers.Reshape(target_shape=(4, 4, 128))
        self.deco1 = tf.keras.layers.Conv2DTranspose(64, 3, strides=2, padding='same',  activation=tf.nn.relu)
        self.deco2 = tf.keras.layers.Conv2DTranspose(32, 3, strides=2, padding='same',  activation=tf.nn.relu)
        # No activation
        self.deco3 = tf.keras.layers.Conv2DTranspose(img_chn, 3, strides=2, padding='same')
        
    def call(self, inp):
        x = self.dens3(inp)
        x = self.resh1(x)
        x = self.deco1(x)
        x = self.deco2(x)
        x = self.deco3(x)
        return x
    
    def model(self):
        x = Input(shape=(2))
        return Model(inputs=[x], outputs=self.call(x))

In [10]:
class AE(tf.keras.Model):
    def __init__(self,hp):
        super(AE, self).__init__()
        self.encoder = Encoder(hp)
        self.decoder = Decoder()

    def call(self, inp):
        mu, va, z = self.encoder(inp)
        out_decoder = self.decoder(z)
        return z, out_decoder 
    
    def model(self):
        x = Input(shape=(32,32,1))
        return Model(inputs=[x], outputs=self.call(x))
    
    def encode(self, img):
        return self.encoder(img)
    
    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits
    
    @tf.function
    def sample(self, z=None):
        if z is None:
            z = tf.random.normal(shape=(100, latent_dim))
        return self.decode(z, apply_sigmoid=True)
    
    @tf.function
    def train_step(self, x, optimizer):
        """Executes one training step and returns the loss.

        This function computes the loss and gradients, and uses the latter to
        update the model's parameters.
        """
        with tf.GradientTape() as tape:
            loss = compute_loss(model, x)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [11]:
class CVAE_Hypermodel(keras_tuner.HyperModel):
    def build(self, hp):
        return AE(hp)
    
    

In [12]:

h_model = CVAE_Hypermodel()
hp = keras_tuner.HyperParameters()
model = h_model.build(hp)

In [13]:
import time
tf.config.run_functions_eagerly(False)
plot_latent_images(model, 10, epoch=0)


optimizer = tf.keras.optimizers.Adam(1e-4)

for epoch in range(1, epochs + 1):
    start_time = time.time()
    for idx, train_x in enumerate(train_dataset):
        model.train_step(train_x, optimizer)
        if epoch == 1 and idx % 75 == 0:
            plot_latent_images(model, 10, epoch=epoch, first_epoch=True, f_ep_count=idx)          
    end_time = time.time()
    loss = tf.keras.metrics.Mean()
    for test_x in test_dataset:
        loss(compute_loss(model, test_x))
    elbo = -loss.result()
    #display.clear_output(wait=False)
    print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
        .format(epoch, elbo, end_time - start_time))
    if epoch != 1:
        plot_latent_images(model, 20, epoch=epoch)


In compute loss
In compute loss


<ipython-input-5-39a7583800ff>:27: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(10, 10))


In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compute loss
In compu

KeyboardInterrupt: 

In [ ]:
train_dataset